In [ ]:
from distributed import Client

client = Client()
client

In [ ]:
import dask.array as da
import intake
import numpy as np
import xarray as xr
import xdggs  # noqa: F401

In [ ]:
cat = intake.open_catalog(
    "https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml"
)["online"]
cat

In [ ]:
level = 15
ds = (
    cat.nicam_220m_test(zoom=level, chunks={"cell": "auto"})
    .to_dask()
    .squeeze()
    .assign(windspeed=lambda ds: np.hypot(ds["ss_u10m"], ds["ss_v10m"]))
)
ds

In [ ]:
decoded = ds.assign_coords(
    {
        "cell_ids": (
            "cell",
            da.arange(
                12 * 4**level, dtype="uint64", chunks=ds["windspeed"].data.chunks[0][0]
            ),
        )
    }
).dggs.decode(
    {"grid_name": "healpix", "level": level, "indexing_scheme": "nested"},
    index_kind="moc",
)
decoded

manually subset the dataset, based on a bounding box:

In [ ]:
box = [125.8, 29, 151, 55]
polygon = np.array(
    [
        [box[0], box[1]],
        [box[2], box[1]],
        [box[2], box[3]],
        [box[0], box[3]],
    ]
)

In [ ]:
import cdshealpix
from astropy.coordinates import Latitude, Longitude

In [ ]:
lon = Longitude(polygon[:, 0], unit="degree")
lat = Latitude(polygon[:, 1], unit="degree")

In [ ]:
cell_ids, _, _ = cdshealpix.nested.polygon_search(
    lon, lat, depth=decoded.dggs.grid_info.level, flat=True
)
cell_ids

In [ ]:
slice_ = slice(cell_ids.min().item(), cell_ids.max().item())
slice_

In [ ]:
subset = decoded.isel(cell=slice_)
subset

In [ ]:
parents = subset.dggs.zoom_to(level=12)
parents

In [ ]:
parents_ = parents.chunk(subset.chunksizes)
parents_

In [ ]:
grid_info = subset.dggs.grid_info.to_dict() | {"level": 12}
downsampled = (
    subset.assign_coords(parents=parents_)
    .drop_indexes("cell_ids")
    .groupby(parents=xr.groupers.UniqueGrouper(labels=np.unique(parents)))
    .mean()
    .rename_vars({"parents": "cell_ids"})
    .rename_dims({"parents": "cells"})
    .dggs.decode(grid_info)
)
downsampled

In [ ]:
computed = downsampled["windspeed"].compute()

In [ ]:
computed.dggs.explore(alpha=0.8)